# Overall comparison of the models' performance (New Metrics)

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import dill
import os


## Printing 1

In [ ]:
def printing(name,data,categories,boxnames):

     temp = pd.DataFrame(data.transpose(),columns=boxnames,index=categories)
     print(name)
     display(temp)
     

## Printing 2 (Anually)

In [ ]:
def printing2(j,name,targets,predictions,years,categories,boxnames):

     temp = pd.DataFrame(predictions.transpose(),columns=years,index=categories)
     temp.loc['targets']=(targets.transpose())
     print(name+ ' ' + boxnames[j])
     display(temp)
     

## Plotting 1

In [ ]:
def plotting (name,quantity,categories,boxnames):

    for i in range (0,len(categories)):
        plt.plot(quantity[:,i],marker= '*', label=categories[i])

    plt.xlabel('Regions')
    plt.xticks(ticks=np.arange(0,len(quantity)), labels=boxnames)
    plt.xticks(rotation=45)
    plt.suptitle(name)
    plt.legend()
    plt.show()


## Plotting 2 (Anually)

In [ ]:
def plotting2(name,targets,predictions,years,categories,boxnames):

    for j in range (0,len(predictions)):

        printing2(j,'Summation',targets[j,:,0],predictions[j],years,categories,boxnames)

        plt.plot(years,targets[j,:,0],marker = '*', label='targets',linewidth=1.5,color='k')
        plt.plot(years,predictions[j],marker= '*', label=categories)
        plt.xlabel('Years')
        plt.suptitle(name+ ' ' + boxnames[j])
        plt.legend()
        plt.show()


## Plotting Mean Values

In [ ]:
def plotting_mean_values(ticks,years,targets,predictions,categories,boxnames,type):

    targets_masked = np.ma.array(targets)

    predictions_masked = np.ma.array(predictions)

    for year in years[:-1]:
        targets_masked[ticks] = np.ma.masked
        predictions_masked[ticks] = np.ma.masked

    for i in range (0,targets.shape[-1]):

        fig, _ = plt.subplots(figsize=(19,5))

        plt.plot(targets_masked[:,i], label = 'targets', linewidth=1.5,color='k')

        for j in range (0, len(categories)):
            plt.plot(predictions_masked[:,i,j], label = categories[j])
            plt.xlabel('Years')
            plt.xticks(ticks,years)
            plt.suptitle('Mean Concentrations [mmol m-2] '+ boxnames[i]+ ' ' +type)
            plt.legend()

        plt.show()


## Initialization

In [ ]:
# Paths
diat = '/data/ibougoudis/MOAD/files/results/Diatom/'
flag = '/data/ibougoudis/MOAD/files/results/Flagellate/'

years = np.array((2021,2022,2023,2024))

variable = diat

categories0 = os.listdir(variable) # All of them
print(categories0)

categories = ['func_reg_boxes_s1','func_reg_boxes_s17','func_reg_boxes_s18','func_reg_boxes_s20'] # What we actually want

boxnames = ['SoG_north','SoG_center','Fraser_plume','SoG_south', 'Haro_Boundary', 'JdF_west', 'JdF_east', 'PS_all', 'PS_main']

# Only to obtain dimensions - number of boxes
with open(variable+categories[0] + '/targets-predictions.pkl', 'rb') as f:
    metrics = dill.load(f)

temp = metrics[0]

r_train = np.zeros((temp.shape[2],len(categories)))
rms_train = np.zeros((temp.shape[2],len(categories)))
slope_train = np.zeros((temp.shape[2],len(categories)))
r_train_season = np.zeros((temp.shape[2],len(categories)))
slope_train_season = np.zeros((temp.shape[2],len(categories)))

r_test = np.zeros((temp.shape[2],len(categories)))
rms_test = np.zeros((temp.shape[2],len(categories)))
slope_test = np.zeros((temp.shape[2],len(categories)))

r_test_season = np.zeros((temp.shape[2],len(categories)))
slope_test_season = np.zeros((temp.shape[2],len(categories)))

targets_sum = np.zeros((temp.shape[2],len(years),len(categories)))
predictions_sum = np.zeros((temp.shape[2],len(years),len(categories)))

targets_mean =np.zeros((temp.shape[2],len(years),len(categories)))
predictions_mean = np.zeros((temp.shape[2],len(years),len(categories)))

targets_diff = np.zeros((temp.shape[0],temp.shape[1],temp.shape[2],len(categories)))
predictions_diff = np.zeros((temp.shape[0],temp.shape[1],temp.shape[2],len(categories)))

rss = np.zeros((temp.shape[2],len(categories)))

predictions = np.zeros((temp.shape[0],temp.shape[1],temp.shape[2],len(categories)))


## Variable Loading

In [ ]:
for i in range (0, len(categories)):

    with open(variable+categories[i] + '/train_metrics.pkl', 'rb') as f:
        metrics = dill.load(f)

    r_train[:,i] = metrics[0]
    rms_train[:,i] = metrics[1]
    slope_train[:,i] = metrics[2]

    r_train_season[:,i] = metrics[3]
    slope_train_season[:,i] = metrics[4]

    season = metrics[5]

    with open(variable+categories[i] + '/test_metrics.pkl', 'rb') as f:
        metrics = dill.load(f)

    r_test[:,i] = metrics[0]
    rms_test[:,i] = metrics[1]
    slope_test[:,i] = metrics[2]
    r_test_season[:,i] = metrics[3]
    slope_test_season[:,i] = metrics[4]

    targets_sum[:,:,i] = metrics[5]
    predictions_sum[:,:,i] = metrics[6]
    targets_mean[:,:,i] = metrics[7]
    predictions_mean[:,:,i] = metrics[8]

    targets_diff = metrics[9]
    predictions_diff[:,:,:,i] = metrics[10]

    rss[:,i] = metrics[11]

    with open(variable+categories[i] + '/targets-predictions.pkl', 'rb') as f:
        metrics = dill.load(f)

    targets = metrics[0]
    predictions[:,:,:,i] = metrics[1]

categories = ['default','no_air_temp','no_wind_s','no_wind_s+spec_hum'] # Names that we want


## Time-series Calculations

In [ ]:
targets_season = np.zeros(targets.shape)
predictions_season = np.zeros(predictions.shape)

for i in range (0,targets.shape[1]):
    targets_season[:,i,:] = targets[:,i,:] -season.transpose()

    for j in range (0, len(categories)):
        predictions_season[:,i,:,j] = predictions[:,i,:,j] -season.transpose()

ticks = []
for i in range (0,targets.shape[0]*targets.shape[1],targets.shape[0]):
    ticks.append(i)

targets_season_new = np.reshape(targets_season,(temp.shape[0]*temp.shape[1],temp.shape[-1]),order='F')
predictions_season_new = np.reshape(predictions_season,(predictions.shape[0]*predictions.shape[1],predictions.shape[2],predictions.shape[3]), order='F')
                       
targets_diff_new = np.reshape(targets_diff,(targets_diff.shape[0]*targets_diff.shape[1],targets_diff.shape[2]), order='F')
predictions_diff_new = np.reshape(predictions_diff,(predictions_diff.shape[0]*predictions_diff.shape[1],predictions_diff.shape[2],predictions_diff.shape[3]), order='F')


## Training Metrics

In [ ]:
printing('Correlation Coefficient (Training)',r_train,categories,boxnames)
plotting('Correlation Coefficient (Training)',r_train,categories,boxnames)

printing('Root Mean Square Error (Training)',rms_train,categories,boxnames)
plotting('Root Mean Square Error (Training)',rms_train,categories,boxnames)

printing('Slope of the Best Fitting Line (Training)',slope_train,categories,boxnames)
plotting('Slope of the Best Fitting Line (Training)',slope_train,categories,boxnames)

printing('Correlation Coefficient (Training, no Seasonality)',r_train_season,categories,boxnames)
plotting('Correlation Coefficient (Training, no Seasonality)',r_train_season,categories,boxnames)

printing('Slope of the Best Fitting Line (Training, no Seasonality)',slope_train_season,categories,boxnames)
plotting('Slope of the Best Fitting Line (Training, no Seasonality)',slope_train_season,categories,boxnames)


## Testing Metrics

In [ ]:
printing('Correlation Coefficient (Testing)',r_test,categories,boxnames)
plotting('Correlation Coefficient (Testing)',r_test,categories,boxnames)

printing('Root Mean Square Error (Testing)',rms_test,categories,boxnames)
plotting('Root Mean Square Error (Testing)',rms_test,categories,boxnames)

printing('Slope of the Best Fitting Line (Testing)',slope_test,categories,boxnames)
plotting('Slope of the Best Fitting Line (Testing)',slope_test,categories,boxnames)

printing('Correlation Coefficient (Testing, no Seasonality)',r_test_season,categories,boxnames)
plotting('Correlation Coefficient (Testing, no Seasonality)',r_test_season,categories,boxnames)

printing('Slope of the Best Fitting Line (Testing, no Seasonality)',slope_test_season,categories,boxnames)
plotting('Slope of the Best Fitting Line (Testing, no Seasonality)',slope_test_season,categories,boxnames)

printing('Sum of Squared Residuals (Testing)',rss,categories,boxnames)
plotting('Sum of Squared Residuals (Testing)',rss,categories,boxnames)


## Annual Metrics

In [60]:
plotting2('Summation',targets_sum,predictions_sum,years,categories,boxnames)

plotting2('Mean',targets_mean,predictions_mean,years,categories,boxnames)

plotting_mean_values(ticks,years,targets_season_new,predictions_season_new,categories,boxnames, '')

plotting_mean_values(ticks,years,targets_diff_new,predictions_diff_new,categories,boxnames, '(Differences)')
